[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dbamman/anlp25/blob/main/7.lm/HW6_Perplexity.ipynb)

# HW6: Perplexity

In this homework, you will implement a function to calculate the perplexity of the n-gram language models we covered in lab, and experiment with different sequences to better understand both n-gram LMs as well as the perplexity metric.

In [ ]:
import copy
from collections import Counter

import nltk
import numpy as np
from nltk import sent_tokenize, word_tokenize

nltk.download("punkt")
nltk.download("punkt_tab")

In [ ]:
def read_file(filename):
    sequences = []
    with open(filename) as file:
        data = file.read()
        sents = sent_tokenize(data)
        for sent in sents:
            tokens = word_tokenize(sent)
            sequences.append(tokens)
    return sequences

In [ ]:
!wget --no-check-certificate https://raw.githubusercontent.com/dbamman/anlp25/refs/heads/main/data/1342_pride_and_prejudice.txt

In [ ]:
# Read data from file and tokenize them into sequences comprised of tokens.

# Pride and Prejudice (Jane Austen)
sequences = read_file("1342_pride_and_prejudice.txt")

max_sequences = 10000

In [ ]:
class NgramModel():

    def __init__(self, sequences, order):

        # For this exercise we're going to encode the LM as a sparse dictionary (trading less storage for more compute)
        # We'll store the LM as a dictionary with the conditioning context as keys; each value is a
        # Counter object that keeps track of the number of times we see a word following that context.
        self.counts = {}

        # Markov order (order 1 = conditioning on previous 1 word; order 2 = previous 2 words, etc.)
        self.order = order

        vocab = {"[END]": 0}

        for s_idx, tokens in enumerate(sequences):
            # We'll add [START] and [END] tokens to encode the beginning/end of sentences
            tokens = ["[START]"] * order + tokens + ["[END]"]

            if s_idx == 0:
                print(tokens)

            for i in range(order, len(tokens)):
                context = " ".join(tokens[i - order:i])
                word = tokens[i]

                if word not in vocab:
                    vocab[word] = len(vocab)

                # For just the first sentence, print the conditioning context + word
                if s_idx == 0:
                    print("Context: %s Next: %s" % (context.ljust(50), word))

                if context not in self.counts:
                    self.counts[context] = Counter()
                self.counts[context][word] += 1



    def sample(self, context):
        total = sum(self.counts[context].values())

        dist = []
        vocab = []

        # Create a probability distribution for each conditioning context, over the vocab that we've observed it with.
        for idx, word in enumerate(self.counts[context]):
            prob = self.counts[context][word]/total
            dist.append(prob)
            vocab.append(word)

        index = np.argmax(np.random.multinomial(1, pvals=dist))
        return vocab[index]

    def generate_sequence(self, keep_ends=True):
        generated = ["[START]"] * (self.order)
        word = None
        while word != "[END]":
            context = ' '.join(generated[-self.order:] if self.order > 0 else "")
            word = self.sample(context)
            generated.append(word)
        if not keep_ends:
            generated = generated[self.order:-1]
        return " ".join(generated)



Let's create some language models of different orders from *Pride and Prejudice*.

In [ ]:
ngram0 = NgramModel(sequences[:max_sequences], order=0)

In [ ]:
ngram1 = NgramModel(sequences[:max_sequences], order=1)

In [ ]:
ngram2 = NgramModel(sequences[:max_sequences], order=2)

**Q1.** Create a `perplexity` function that can takes two arguments: a.) a model of *any* ngram order (from the class above); and b.) a sequence to calculate perplexity for.  You'll recall from class that perplexity under a particular language model for sequence $w$ is given by the following equation:

$$
\textrm{perplexity}_{model}(w) = \exp\left(-{1 \over N} \sum_{i=1}^N \log P_{model}(w_i) \right)
$$

$P_{model}(w_i)$ calculates the probability of token $w_i$ using whatever assumptions that model makes -- for a bigram model (order 1), this is $P(w_i \mid w_{i-1})$, for a trigram model (order 2), this is $P(w_i \mid w_{i-2}, w_{i-1})$, etc.  Two things to note:

* When calculating the probability of the first word(s), be sure to get the conditioning context right.  The conditioning context for the first word in a trigram model, for example, is $P(w_i \mid$ [START] [START]$)$.
* Perplexity is only calculated for the words in the actual sequence.  We don't include the $P$([START]) or $P$([END]) in the perplexity calculuation.


*Hint*: when working on this function, you might want to debug by printing out the probabilities of each $w_i$.

In [ ]:
from math import log

def perplexity(model, tokens):
    # YOUR CODE HERE

**Q2**. Execute that perplexity function for the following language models:

In [ ]:
perplexity(ngram0, word_tokenize("She was a great friend of Mr. Bingley."))

In [ ]:
perplexity(ngram1, word_tokenize("She was a great friend of Mr. Bingley."))

In [ ]:
perplexity(ngram2, word_tokenize("She was a great friend of Mr. Bingley."))

**Q3.** What is the perplexity of "She was a really great friend of Mr. Bingley." in the trigram language model trained above?

Explain in 100 words what behavior is expected (and correct) given how an n-gram language model works and the data we are training it on.

In [ ]:
perplexity(ngram2, word_tokenize("She was a really great friend of Mr. Bingley."))

**Q4.** What 1-token sequence yields the lowest perplexity for the 0-order n-gram model? Why?

In [ ]:
perplexity(ngram0, word_tokenize("YOUR_ANSWER_HERE"))

**Q5.** Write a function to find the n-token sequence with the lowest perplexity given an n-gram model. Explain why it should work.

In [ ]:
def min_ppl_sequence(model, n=5):
    """Return an n-token long string that yields the lowest possible perplexity under the provided model."""

In [ ]:
min_ppl_sequence(ngram1, n=3)

In [ ]:
perplexity(ngram1, min_ppl_sequence(ngram1, n=3))